In [1]:
%matplotlib qt5 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from fastdtw import fastdtw

In [2]:
word = "puzzle" 
data = pd.read_csv("train.csv")
filtered_data = data[data["sign"] == word]
csv_path = "filter_for_word.csv"
filtered_data.to_csv(csv_path, index=False)

In [3]:
def process_path_data(path_number, filtered_data):
    """
    Procesa los datos de una ruta específica (los .parquet), extrae y reformatea la información relevante de los puntos (x, y).

    Args:
        path_number (int): El índice de la ruta en el DataFrame filtered_data.
        filtered_data (pandas.DataFrame): Un DataFrame que contiene información sobre las rutas de los archivos de datos.

    Returns:
        numpy.ndarray: Un array de la secuencia reformateada con la forma (num_samples, 543, 2), donde num_samples es el
                       número de fotogramas únicos en los datos.
    """
    path_file = filtered_data.iloc[path_number]["path"]
    data = pd.read_parquet(path_file)
    # data = data[data["type"] != "face"]
    cleaned_data = data[["frame", "x", "y"]]
    num_samples = cleaned_data["frame"].nunique()
    cleaned_data = cleaned_data[["x", "y"]].to_numpy()
    seq = cleaned_data.reshape(num_samples, 543, 2)
    return seq

seq1 = process_path_data(4, filtered_data)
seq2 = process_path_data(10, filtered_data)

ValueError: cannot reshape array of size 900 into shape (6,543,2)

In [ ]:
def interpolate_sequence(seq, num_frames):
    """
    Interpola una secuencia única a una longitud específica en la dimensión del tiempo.
    
    Args:
        seq (numpy.ndarray): La secuencia de entrada con la forma (T, 543, 2), donde T es el número de fotogramas.
        num_frames (int): El número de fotogramas para la secuencia interpolada.

    Returns:
        numpy.ndarray: La secuencia interpolada con la forma (num_frames, 543, 2).
    """
    seq_interp = np.zeros((num_frames, 75, 2))
    seq = np.nan_to_num(seq)
    
    for i in range(75):
        for j in range(2):
            f = interp1d(np.arange(seq.shape[0]), seq[:, i, j], kind='linear')
            seq_interp[:, i, j] = f(np.linspace(0, seq.shape[0] - 1, num_frames))

    return seq_interp

def interpolate_sequences_list(sequences, num_frames):
    """
    Interpola una lista de secuencias a una longitud específica en la dimensión del tiempo.
    
    Args:
        sequences (list): Una lista de secuencias, cada una con la forma (T, 543, 2), donde T es el número de fotogramas.
        num_frames (int): El número de fotogramas para las secuencias interpoladas.

    Returns:
        numpy.ndarray: Un array que contiene las secuencias interpoladas con la forma (num_frames, 543, 2, num_sequences),
                       donde num_sequences es el número de secuencias en la lista de entrada.
    """
    num_sequences = len(sequences)
    interpolated_sequences = np.zeros((num_frames, 75, 2, num_sequences))
    
    for idx, seq in enumerate(sequences):
        seq_interp = interpolate_sequence(seq, num_frames)
        interpolated_sequences[:, :, :, idx] = seq_interp
    
    return interpolated_sequences

num_frames = 30 # Número de fotogramas para las secuencias interpoladas
sequences = [] # Lista de secuencias
for i in range(filtered_data.shape[0]):
    seq = process_path_data(i, filtered_data)
    sequences.append(seq)

interpolated_sequences = interpolate_sequences_list(sequences, num_frames)

In [ ]:
""" export interpolated_sequences to npy"""
np.save(f"interpolated_sequences_{word}_75points.npy", interpolated_sequences)

In [ ]:
""" visualizar datos"""

data_descriptors_rendering = np.load(f"interpolated_sequences_{word}_75points.npy")

plt.plot(data_descriptors_rendering[1, :, 0], data_descriptors_rendering[1, :, 1], 'o')
